# Test of GAT
- use DGL
- predict `graphs`
- test: 0~99
- validation: 100~199
- train: 200~999
- larger lr with scheduler

In [1]:
import os
import dgl
import json
import torch
import torch as th
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

- check the GPU and assign the GPU by the best memory usage

In [2]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

print(device)


cuda:1


## Fix the seed

In [3]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [4]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]

        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)

        g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
        g.edata['feat'] = th.tensor(data["edge_attr"]).to(self.device)  # Add edge features to graph

        return g, th.tensor(data["label"]).to(self.device)


def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


In [5]:
datasets = ['train', 'valid', 'test']
dataset_data = {}

for dataset_name in tqdm(datasets):
#     file_path = f"../../data_processing/dgl/data/test_graph/repeated_{dataset_name}.jsonl"
    file_path = f"../../data_processing/dgl/data_new/training_data/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")

  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/training_data/train.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/training_data/valid.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/training_data/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [6]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(4)
# dataloaders = create_dataloaders(16)

- Turn the print message to a log file

In [8]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GAT_data_graph_4.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/0826_11:32_GAT_data_graph_4.log


### Model

In [9]:
class GAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads, dropout_prob=0.25):
        super(GAT, self).__init__()
        
        # do not check the zero in_degree since we have all the complete graph
        self.layer1 = GATConv(in_dim, hidden_dim, num_heads=num_heads, activation=F.relu, allow_zero_in_degree=True)
        self.layer2 = GATConv(hidden_dim * num_heads, out_dim, num_heads=num_heads, allow_zero_in_degree=True)
        
        # Adding Batch Normalization after each GAT layer
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim * num_heads)
        self.batchnorm2 = nn.BatchNorm1d(out_dim)
        
        # Adding Dropout for regularization
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, g, h):
        # Apply GAT layers
        h = self.layer1(g, h)
        h = h.view(h.shape[0], -1)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h).squeeze(1)
        
        # Store the output as a new node feature
        g.ndata['h_out'] = h

        # Use mean pooling to aggregate this new node feature
        h_agg = dgl.mean_nodes(g, feat='h_out')
        return h_agg

    

- Model Forward  

In [10]:
def model_fn(data, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
    batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = labels.to(device)
    logits = model(batched_g, batched_g.ndata['feat'].float()) # for GAT
    logits = logits.mean(dim=1)
    
    loss = criterion(logits, labels)

    # Get the class id with the highest probability
    preds = logits.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 5000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [11]:
seed = 8787
same_seeds(seed)

model = GAT(in_dim=50, hidden_dim=16, out_dim=168, num_heads=8)
torch.save(model.state_dict(), 'model1_initial/initial_weight.pth')

In [12]:
model.layer1.fc.weight

Parameter containing:
tensor([[-0.1806, -0.0598,  0.0091,  ...,  0.0719,  0.2496,  0.0873],
        [ 0.1694, -0.0015, -0.0139,  ...,  0.0147,  0.0892,  0.0146],
        [ 0.0969, -0.0595, -0.0115,  ..., -0.0474,  0.0529, -0.0565],
        ...,
        [-0.0433, -0.2248,  0.3002,  ...,  0.0850,  0.1621,  0.0422],
        [ 0.2097, -0.2492,  0.0612,  ..., -0.0041,  0.0365, -0.1483],
        [ 0.0971, -0.2221,  0.1652,  ..., -0.1312, -0.2610,  0.0077]],
       requires_grad=True)

- Check if model really load the model_dict

In [13]:
model = GAT(in_dim=50, hidden_dim=16, out_dim=168, num_heads=8)
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))
model.layer1.fc.weight

Parameter containing:
tensor([[-0.1806, -0.0598,  0.0091,  ...,  0.0719,  0.2496,  0.0873],
        [ 0.1694, -0.0015, -0.0139,  ...,  0.0147,  0.0892,  0.0146],
        [ 0.0969, -0.0595, -0.0115,  ..., -0.0474,  0.0529, -0.0565],
        ...,
        [-0.0433, -0.2248,  0.3002,  ...,  0.0850,  0.1621,  0.0422],
        [ 0.2097, -0.2492,  0.0612,  ..., -0.0041,  0.0365, -0.1483],
        [ 0.0971, -0.2221,  0.1652,  ..., -0.1312, -0.2610,  0.0077]],
       requires_grad=True)

### test of valid and test part is ``graph``

- Batch size = 4
- use large lr and scheduler

In [ ]:
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = GAT(in_dim=50, hidden_dim=16, out_dim=168, num_heads=8)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))
best_model_path = "../checkpoint_GAT/best_model_GAT_4.pt"

model = model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = AdamW(model.parameters(), lr=1e-3)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=18, num_training_steps=total_steps)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)


criterion = nn.CrossEntropyLoss()
total_steps = 360

# save the best model
best_val_loss = float('inf')
patience = 10  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
#     count = 0 
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
#         count += 1
        num_batches += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

        
    scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for data in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)

            torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, best_model_path)

        else:
            waiting += 1
            if waiting >= patience:
                add_log_msg("Early stopping")
                break


# Testing Part
model.eval()
total = 0
correct = 0
count = 0

with torch.no_grad():
    for data in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):

        loss, accuracy, predicted = model_fn(data, model, criterion, device, count, which_type='test')
        labels = data[1].to(device)  # Assuming labels are the second element in the tuple
        
        add_log_msg(f"labels: {labels} {labels.shape}")
        add_log_msg(f"predicted: {predicted} {predicted.shape}")
        count += 1
        
        total += labels.size(0) # label.size(0) is the batch size
        correct += (predicted == labels).sum().item() 
        # (predicted == labels).sum() would return how many of them are equal; 
        # .item() would make the tensor to the regular value
        
#     print('Test Accuracy: %d %%' % (100 * correct / total))
add_log_msg(f'Test Accuracy: {100 * correct / total} %%')

  0%|          | 0/360 [00:00<?, ?it/s]

Training:   0%|          | 0/33000 [00:00<?, ?it/s]

08/26/2023, 11:34:00# labels of 5000: tensor([107,  73,  21, 144], device='cuda:1') torch.Size([4])
08/26/2023, 11:34:00# predicted of 5000: tensor([ 66, 115, 132,  66], device='cuda:1') torch.Size([4])
08/26/2023, 11:35:16# labels of 10000: tensor([ 74,  12, 161,  81], device='cuda:1') torch.Size([4])
08/26/2023, 11:35:16# predicted of 10000: tensor([ 38, 131, 161, 131], device='cuda:1') torch.Size([4])
08/26/2023, 11:36:32# labels of 15000: tensor([143,  67,  31, 132], device='cuda:1') torch.Size([4])
08/26/2023, 11:36:32# predicted of 15000: tensor([132,  78, 111,  28], device='cuda:1') torch.Size([4])
08/26/2023, 11:37:48# labels of 20000: tensor([162,  59, 165, 101], device='cuda:1') torch.Size([4])
08/26/2023, 11:37:48# predicted of 20000: tensor([ 68, 132, 165, 132], device='cuda:1') torch.Size([4])
08/26/2023, 11:39:03# labels of 25000: tensor([ 15,  95,  73, 140], device='cuda:1') torch.Size([4])
08/26/2023, 11:39:03# predicted of 25000: tensor([ 15,  95,  73, 140], device='cu

Validation:   0%|          | 0/4125 [00:00<?, ?it/s]

08/26/2023, 11:41:05# labels of Validation: tensor([134,  42,  29,  24], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:05# predicted of Validation: tensor([134, 148,  29, 148], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:05# labels of 0: tensor([134,  42,  29,  24], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:05# predicted of 0: tensor([134, 148,  29, 148], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:14# labels of Validation: tensor([ 46,  39, 146, 148], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:14# predicted of Validation: tensor([148,  39, 146,   6], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:25# labels of Validation: tensor([47, 86, 38, 21], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:25# predicted of Validation: tensor([148, 148, 148,   6], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:35# labels of Validation: tensor([166, 107,  76, 147], device='cuda:1') torch.Size([4])
08/26/2023, 11:41:35# predicted of Validation: tensor([ 

...

Training:   0%|          | 0/33000 [00:00<?, ?it/s]

08/26/2023, 18:43:04# labels of 5000: tensor([ 49, 162,  23,   1], device='cuda:1') torch.Size([4])
08/26/2023, 18:43:04# predicted of 5000: tensor([ 47,  47,  23, 137], device='cuda:1') torch.Size([4])
08/26/2023, 18:44:23# labels of 10000: tensor([ 28, 105,  37,  63], device='cuda:1') torch.Size([4])
08/26/2023, 18:44:23# predicted of 10000: tensor([132, 105,  59,  43], device='cuda:1') torch.Size([4])
08/26/2023, 18:45:37# labels of 15000: tensor([128, 153, 105,  33], device='cuda:1') torch.Size([4])
08/26/2023, 18:45:37# predicted of 15000: tensor([128,   9, 105,  38], device='cuda:1') torch.Size([4])
08/26/2023, 18:46:51# labels of 20000: tensor([158, 126,  20,  49], device='cuda:1') torch.Size([4])
08/26/2023, 18:46:51# predicted of 20000: tensor([ 44, 126,  30,  30], device='cuda:1') torch.Size([4])
08/26/2023, 18:48:07# labels of 25000: tensor([75, 88, 98, 73], device='cuda:1') torch.Size([4])
08/26/2023, 18:48:07# predicted of 25000: tensor([34, 66,  4, 73], device='cuda:1') t

Validation:   0%|          | 0/4125 [00:00<?, ?it/s]

08/26/2023, 18:50:13# labels of Validation: tensor([143, 107,  62,  73], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:13# predicted of Validation: tensor([49, 28, 62, 73], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:13# labels of 0: tensor([143, 107,  62,  73], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:13# predicted of 0: tensor([49, 28, 62, 73], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:22# labels of Validation: tensor([96, 93,  0, 52], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:22# predicted of Validation: tensor([96, 28,  0, 52], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:31# labels of Validation: tensor([129,   2,  76,   4], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:31# predicted of Validation: tensor([28, 49, 49, 49], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:41# labels of Validation: tensor([123, 138, 112,  49], device='cuda:1') torch.Size([4])
08/26/2023, 18:50:41# predicted of Validation: tensor([123, 138,  49,  4

Training:   0%|          | 0/33000 [00:00<?, ?it/s]

In [ ]:
# device = torch.device(f"cuda:2")
# print(device)